In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Head_and_Neck_Cancer/GSE244580'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data of tonsills of chronic tonsillitis, and peri-tumoral tonsillar tissues and lymph nodes in oropharyngeal cancer"
!Series_summary	"Secondary lymphoid organs (SLOs) serve as an interface between tumor cells and immune system as an initial site of antigen presentation which is critical for the development of an effective anti-tumor immune response."
!Series_summary	"We used microarray to analyze the gene expression profile of tonsills from chronic tonsillitis, and peri-tumoral tonsillar tissues and lymph nodes of oropharyngeal cancer."
!Series_overall_design	"We analyzed and compared the profiles of these organs."
Sample Characteristics Dictionary:
{0: ['disease state: chronic tonsillitis', 'disease state: peritumoral tonsillar tissue', 'disease state: lymph node of oropharyngeal cancer']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # Based on the background information provided

# Locate the keys for variables in the Sample Characteristics Dictionary
sample_characteristics_dict = {
    0: ['disease state: chronic tonsillitis', 'disease state: peritumoral tonsillar tissue', 'disease state: lymph node of oropharyngeal cancer']
}

trait_row = 0  # The disease state information seems to be in key 0

# No information regarding age and gender is available based on the provided dictionary
# Hence, age_row and gender_row are set to None explicitly
age_row = gender_row = None  # No age and gender data available

# Functions to convert the data types
def convert_trait(value):
    trait_mapping = {
        "chronic tonsillitis": 0,
        "peritumoral tonsillar tissue": 1,
        "lymph node of oropharyngeal cancer": 1
    }
    key_value = value.split(': ')[-1].strip()
    return trait_mapping.get(key_value, None)

def convert_age(value):
    try:
        age_value = int(value.split(': ')[-1].strip())
        return age_value
    except ValueError:
        return None

def convert_gender(value):
    gender_mapping = {
        "female": 0,
        "male": 1
    }
    key_value = value.split(': ')[-1].strip().lower()
    return gender_mapping.get(key_value, None)

# Save cohort information
save_cohort_info('GSE244580', './preprocessed/Head_and_Neck_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Head_and_Neck_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Head_and_Neck_Cancer/trait_data/GSE244580.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM7820913': [0], 'GSM7820914': [0], 'GSM7820915': [0], 'GSM7820916': [1], 'GSM7820917': [1], 'GSM7820918': [0], 'GSM7820919': [1], 'GSM7820920': [1], 'GSM7820921': [1], 'GSM7820922': [1], 'GSM7820923': [1], 'GSM7820924': [1], 'GSM7820925': [1], 'GSM7820926': [1], 'GSM7820927': [1], 'GSM7820928': [1], 'GSM7820929': [1], 'GSM7820930': [1], 'GSM7820931': [1], 'GSM7820932': [1], 'GSM7820933': [1], 'GSM7820934': [1], 'GSM7820935': [1], 'GSM7820936': [1], 'GSM7820937': [1], 'GSM7820938': [1], 'GSM7820939': [1], 'GSM7820940': [1], 'GSM7820941': [1], 'GSM7820942': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [12190.0, 29554.0, 69091.0, 160446.0, 317811.0], 'RANGE_END': [13639.0, 31109.0, 70008.0, 161525.0, 328581.0], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'GB_ACC': ['NR_046018', nan, nan, nan, 'NR_024368'], 'SPOT_ID': ['chr1:12190-13639', 'chr1:29554-31109', 'chr1:69091-70008', 'chr1:160446-161525', 'chr1:317811-328581'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'GB_ACC'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping_df = get_gene_mapping(gene_annotation, prob_col=identifier_key, gene_col=gene_symbol_key)

# 3. Apply the mapping and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Head_and_Neck_Cancer/gene_data/GSE244580.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Head_and_Neck_Cancer')

# 4. Save the cohort information.
save_cohort_info('GSE244580', './preprocessed/Head_and_Neck_Cancer/cohort_info.json', True, True, trait_biased, merged_data)

# If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Head_and_Neck_Cancer/GSE244580.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
